In [35]:
from os import walk
import pandas as pd
import numpy as np
import torch
import time
import torch.nn as nn
from progressbar import ProgressBar
from tqdm import tqdm
dataframe_list = pd.read_csv('/home/wuwenjun/Data/AlphaNet_Original_Input.csv')

"""将timestamp转化为unix"""
unix_df = pd.DataFrame(dataframe_list['timestamp'].drop_duplicates())
unix_df["unix"] = unix_df['timestamp'].apply(lambda x: time.mktime(time.strptime(x, '%Y-%m-%d %H:%M:%S')))
dataframe_list = pd.merge(dataframe_list , unix_df, how = 'left', on = 'timestamp')
dataframe_list['timestamp'] = dataframe_list['unix']
dataframe_list.drop('unix',axis=1,inplace=True)
del unix_df
"-----------------------------------"
dataframe_list = dataframe_list.sort_values(by=['ticker','timestamp'])
dataframe_list

,timestamp,ticker,pre_close,open,high,low,close,vwap,volume,pct_chg,turnover,free_turnover,chg,5d_ret
0,1.420508e+09,1,15.84,15.99,16.28,15.60,16.02,15.9605,2860436.43,1.1364,2.9079,5.7657,0.18,-0.075710
1,1.420508e+09,2,13.90,14.39,15.29,14.22,14.91,14.7858,6560835.70,7.2662,6.7834,8.1741,1.01,-0.106164
2,1.420508e+09,4,15.57,15.58,15.90,15.30,15.69,15.5709,28095.12,0.7707,3.3496,4.5343,0.12,0.006390
3,1.420508e+09,6,7.05,7.10,7.39,7.02,7.08,7.1881,1078647.12,0.4255,8.0562,13.2751,0.03,-0.078571
4,1.420508e+09,7,14.60,14.51,14.80,14.25,14.52,14.5105,28415.72,-0.5479,1.3807,1.7538,-0.08,-0.004844
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4851233,1.625103e+09,688699,237.77,285.32,285.32,285.32,285.32,285.3200,1225.82,19.9983,0.6940,0.6940,47.55,-0.024476
4851234,1.625103e+09,688777,90.00,89.95,94.80,89.06,94.15,92.9798,19001.61,4.6111,4.6288,4.6288,4.15,0.053133
4851235,1.625103e+09,688788,138.20,137.28,148.50,136.33,145.38,144.3683,13572.56,5.1954,7.7977,7.7977,7.18,-0.102721
4851236,1.625103e+09,688819,43.68,43.40,44.98,43.31,44.49,44.3727,47249.01,1.8544,4.6754,4.6754,0.81,0.032164


In [90]:
dataframe_list

,timestamp,ticker,pre_close,open,high,low,close,vwap,volume,pct_chg,turnover,free_turnover,chg,5d_ret
0,1.420508e+09,1,15.84,15.99,16.28,15.60,16.02,15.9605,2860436.43,1.1364,2.9079,5.7657,0.18,-0.075710
2257,1.420594e+09,1,16.02,15.85,16.39,15.55,15.78,15.9408,2166421.40,-1.4981,2.2024,4.3668,-0.24,-0.050129
4524,1.420681e+09,1,15.78,15.56,15.83,15.30,15.48,15.4977,1700120.67,-1.9011,1.7283,3.4269,-0.30,-0.041935
6792,1.420767e+09,1,15.48,15.50,15.57,14.90,14.96,15.1167,1407714.21,-3.3592,1.4311,2.8375,-0.52,0.033557
9060,1.421026e+09,1,14.96,14.90,15.87,14.71,15.08,15.2895,2508500.23,0.8021,2.5501,5.0564,0.12,-0.057835
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4835137,1.624585e+09,688981,58.54,58.53,59.11,57.55,57.70,58.1733,273520.23,-1.4349,2.4964,2.4964,-0.84,0.041032
4839157,1.624844e+09,688981,57.70,57.76,59.99,57.76,59.31,59.2746,474988.89,2.7903,4.3352,4.3352,1.61,-0.039098
4843181,1.624930e+09,688981,59.31,59.85,60.19,58.86,59.06,59.4762,341515.10,-0.4215,3.1170,3.1170,-0.25,-0.014382
4847208,1.625017e+09,688981,59.06,59.10,60.08,58.70,59.25,59.3544,291408.81,0.3217,2.6597,2.6597,0.19,-0.051600


In [88]:
array = []

In [89]:
for i in tqdm(range(0,30)):
    array.append(np.array(dataframe_list.shift(-i)))
array = np.array(array)
array = array.transpose(1,-1,0)
array = array[:-29,:,:]

100%|██████████| 30/30 [00:45<00:00,  1.52s/it]


In [112]:
array.shape

(4851209, 14, 30)

In [161]:
pd.DataFrame(array[300000])

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,1.583804e+09,1.583890e+09,1.583977e+09,1.584063e+09,1.584322e+09,1.584409e+09,1.584495e+09,1.584581e+09,1.584668e+09,1.584927e+09,...,1.586309e+09,1.586396e+09,1.586482e+09,1.586741e+09,1.586828e+09,1.586914e+09,1.587001e+09,1.587087e+09,1.587346e+09,1.587433e+09
1,6.520000e+02,6.520000e+02,6.520000e+02,6.520000e+02,6.520000e+02,6.520000e+02,6.520000e+02,6.520000e+02,6.520000e+02,6.520000e+02,...,6.520000e+02,6.520000e+02,6.520000e+02,6.520000e+02,6.520000e+02,6.520000e+02,6.520000e+02,6.520000e+02,6.520000e+02,6.520000e+02
2,1.263000e+01,1.389000e+01,1.306000e+01,1.291000e+01,1.227000e+01,1.104000e+01,1.214000e+01,1.142000e+01,1.103000e+01,9.950000e+00,...,9.940000e+00,1.048000e+01,1.033000e+01,9.870000e+00,9.850000e+00,1.026000e+01,1.008000e+01,9.810000e+00,9.280000e+00,9.420000e+00
3,1.325000e+01,1.336000e+01,1.221000e+01,1.339000e+01,1.120000e+01,1.140000e+01,1.159000e+01,1.099000e+01,1.060000e+01,9.970000e+00,...,1.020000e+01,1.027000e+01,9.820000e+00,9.850000e+00,9.920000e+00,1.023000e+01,9.810000e+00,9.690000e+00,9.280000e+00,9.300000e+00
4,1.389000e+01,1.468000e+01,1.366000e+01,1.376000e+01,1.188000e+01,1.214000e+01,1.210000e+01,1.193000e+01,1.075000e+01,1.048000e+01,...,1.082000e+01,1.098000e+01,1.023000e+01,1.015000e+01,1.054000e+01,1.031000e+01,1.017000e+01,9.690000e+00,9.580000e+00,9.460000e+00
5,1.290000e+01,1.250000e+01,1.188000e+01,1.218000e+01,1.104000e+01,1.104000e+01,1.094000e+01,1.041000e+01,9.930000e+00,9.970000e+00,...,9.600000e+00,9.800000e+00,9.730000e+00,9.750000e+00,9.750000e+00,9.900000e+00,9.650000e+00,9.200000e+00,9.150000e+00,9.160000e+00
6,1.389000e+01,1.306000e+01,1.291000e+01,1.227000e+01,1.104000e+01,1.214000e+01,1.142000e+01,1.103000e+01,9.950000e+00,1.003000e+01,...,1.048000e+01,1.033000e+01,9.870000e+00,9.850000e+00,1.026000e+01,1.008000e+01,9.810000e+00,9.280000e+00,9.420000e+00,9.350000e+00
7,1.367180e+01,1.319450e+01,1.273320e+01,1.287220e+01,1.122450e+01,1.194450e+01,1.130030e+01,1.107820e+01,1.019460e+01,1.018330e+01,...,1.024180e+01,1.028430e+01,9.952500e+00,9.918000e+00,1.026760e+01,1.004380e+01,9.889100e+00,9.392100e+00,9.347800e+00,9.295200e+00
8,2.234502e+06,4.370255e+06,3.008699e+06,2.609252e+06,2.139863e+06,3.040810e+06,2.656734e+06,2.235745e+06,2.074524e+06,1.523230e+06,...,2.537542e+06,2.466205e+06,1.494843e+06,1.067034e+06,1.783733e+06,1.240929e+06,1.258372e+06,1.352105e+06,1.060823e+06,7.849408e+05
9,9.976200e+00,-5.975500e+00,-1.148500e+00,-4.957400e+00,-1.002440e+01,9.963800e+00,-5.930800e+00,-3.415100e+00,-9.791500e+00,8.040000e-01,...,5.432600e+00,-1.431300e+00,-4.453000e+00,-2.026000e-01,4.162400e+00,-1.754400e+00,-2.678600e+00,-5.402700e+00,1.508600e+00,-7.431000e-01


In [137]:
np.max(array,axis=2) == np.min(array,axis=2)

array([[False,  True, False, ..., False, False, False],
       [False,  True, False, ..., False, False, False],
       [False,  True, False, ..., False, False, False],
       ...,
       [False,  True, False, ..., False, False, False],
       [False,  True, False, ..., False, False, False],
       [False,  True, False, ..., False, False, False]])

In [149]:
max_ticker= np.max(array,axis=2)[:,1].reshape(-1,1)
min_ticker= np.min(array,axis=2)[:,1].reshape(-1,1)

In [159]:
array.shape

(4851209, 14, 30)

In [157]:
array[max_ticker == min_ticker]

IndexError: boolean index did not match indexed array along dimension 1; dimension is 14 but corresponding boolean dimension is 1

In [71]:
array

array([ 1.42050780e+09,  1.00000000e+00,  1.58400000e+01,  1.59900000e+01,
        1.62800000e+01,  1.56000000e+01,  1.60200000e+01,  1.59605000e+01,
        2.86043643e+06,  1.13640000e+00,  2.90790000e+00,  5.76570000e+00,
        1.80000000e-01, -7.57097792e-02])

In [12]:
dataframe_list['timestamp'] = str(pd.to_datetime(dataframe_list['timestamp']))

In [32]:
unix_df = pd.DataFrame(dataframe_list['timestamp'].drop_duplicates())
unix_df["unix"] = unix_df['timestamp'].apply(lambda x: time.mktime(time.strptime(x, '%Y-%m-%d %H:%M:%S')))
unix_df

,timestamp,unix
0,2015-01-06 09:30:00,1.420508e+09
2257,2015-01-07 09:30:00,1.420594e+09
4524,2015-01-08 09:30:00,1.420681e+09
6792,2015-01-09 09:30:00,1.420767e+09
9060,2015-01-12 09:30:00,1.421026e+09
...,...,...
4831124,2021-06-25 09:30:00,1.624585e+09
4835138,2021-06-28 09:30:00,1.624844e+09
4839158,2021-06-29 09:30:00,1.624930e+09
4843182,2021-06-30 09:30:00,1.625017e+09


In [34]:
dataframe_list = pd.merge(dataframe_list , unix_df, how = 'left', on = 'timestamp')
dataframe_list['timestamp'] = dataframe_list['unix']
dataframe_list.drop('unix',axis=1,inplace=True)

,timestamp,ticker,pre_close,open,high,low,close,vwap,volume,pct_chg,turnover,free_turnover,chg,5d_ret,unix
0,1.420508e+09,1,15.84,15.99,16.28,15.60,16.02,15.9605,2860436.43,1.1364,2.9079,5.7657,0.18,-0.075710,1.420508e+09
1,1.420508e+09,2,13.90,14.39,15.29,14.22,14.91,14.7858,6560835.70,7.2662,6.7834,8.1741,1.01,-0.106164,1.420508e+09
2,1.420508e+09,4,15.57,15.58,15.90,15.30,15.69,15.5709,28095.12,0.7707,3.3496,4.5343,0.12,0.006390,1.420508e+09
3,1.420508e+09,6,7.05,7.10,7.39,7.02,7.08,7.1881,1078647.12,0.4255,8.0562,13.2751,0.03,-0.078571,1.420508e+09
4,1.420508e+09,7,14.60,14.51,14.80,14.25,14.52,14.5105,28415.72,-0.5479,1.3807,1.7538,-0.08,-0.004844,1.420508e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4851233,1.625103e+09,688699,237.77,285.32,285.32,285.32,285.32,285.3200,1225.82,19.9983,0.6940,0.6940,47.55,-0.024476,1.625103e+09
4851234,1.625103e+09,688777,90.00,89.95,94.80,89.06,94.15,92.9798,19001.61,4.6111,4.6288,4.6288,4.15,0.053133,1.625103e+09
4851235,1.625103e+09,688788,138.20,137.28,148.50,136.33,145.38,144.3683,13572.56,5.1954,7.7977,7.7977,7.18,-0.102721,1.625103e+09
4851236,1.625103e+09,688819,43.68,43.40,44.98,43.31,44.49,44.3727,47249.01,1.8544,4.6754,4.6754,0.81,0.032164,1.625103e+09


In [28]:
time.mktime(time.strptime("2015-01-06 09:30:00", '%Y-%m-%d %H:%M:%S'))

1420507800.0

In [10]:
str(dataframe_list['timestamp'])

'0         2015-01-06 09:30:00\n1         2015-01-06 09:30:00\n2         2015-01-06 09:30:00\n3         2015-01-06 09:30:00\n4         2015-01-06 09:30:00\n                  ...        \n4851233   2021-07-01 09:30:00\n4851234   2021-07-01 09:30:00\n4851235   2021-07-01 09:30:00\n4851236   2021-07-01 09:30:00\n4851237   2021-07-01 09:30:00\nName: timestamp, Length: 4851238, dtype: datetime64[ns]'